* This notebook is to demonstrate how to use Bayesian Optimization package on github <https://github.com/fmfn/BayesianOptimization> locally to tune hyperparamters for our models(RNN only) for predicting heart failure onset risk on cerner sample data
* For this demonstration, the data is the original 1 hospital (h143) previously used by retain, with 42,729 patients in total
* The hyperparameters to be tuned are: learning rate, l2 regularization, and eps for optimizer, dropout rate, embedding dimension, hidden dimension, number of layers and optimizer
* To implement this, first you need to install the package: however we modify the package file a bit to bypass errors and keep on iterating. The modified files could be found at Experiments/modifiedBO
* Then **important**: you need to define a function (in our case model_tune()) which takes in the hyperparameters: l2_exp, lr_exp, eps_exp on logscale, embed_dim, hid_dim on log2scale, dropout and also ct_code, dlm_code and opt_code, and run the model using models, Loaddata, and TrainVaTe modules and return the best validation auc. We put the categorical parameters: ct_code: cell_type code which includes RNN, GRU and LSTM; dlm_code: model code which includes RNN and DRNN, and opt_code: optimitzer code which includes 7 optimizers to a 3-level loop so that each time we run BO, it takes a combination of model, cell_type and optimizer and return the rest of best parameters based on best validation auc
* Be ware that this BO package will search float parameters, so if you have int or categorical parameters you want to tune, you might want to transform those values in your function before giving those to your models (like we did here)
* Then **important**: call BO function and pass your model_tune(), a search range for each parameter ((-16, 1) means -16 and 1 inclusive), and give it points to explore (points that will give you large target values) if you want to, and call maximize() and pass number of iterations you want to run BO
* Then **important**: if you have an unreliable GPU as we do, you might want to dump previously searched results back to your BO function so that it won't explore those spaces as again. We do that by importing previous searched results stored in 'BORNN_Init.csv' as a dataframe, and call initialize_df() for the Bo object
* Then you will get results of your initial designated explored points(if any), 5 system initializations + previously searched results, and plus number of BO iterations
* For our results:it's time consuming for RNN model, but it improved our best validation auc **from 0.65057** to currently **0.72976**

 Bayesian Optimization
--------------------------------------------------------------------------------------------------------------------------------
Step |   Time |      Value |   dropout |   embdim_exp |   eps_exp |   hid_exp |    l2_exp |   layers_n |    lr_exp |
 
56 | 00m00s |    0.72976 |    0.0000 |       9.0000 |   -9.0000 |    9.0000 |   -3.8164 |     2.0000 |   -5.1497 | 

In [1]:
from __future__ import print_function
from __future__ import division

#from sklearn.datasets import make_classification
#from sklearn.cross_validation import cross_val_score
import string
import re
import random

import os
import sys
import argparse
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torchviz import make_dot, make_dot_from_trace

from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

try:
    import cPickle as pickle
except:
    import pickle


from bayes_opt import BayesianOptimization
import pandas as pd

In [2]:
torch.backends.cudnn.enabled=False

In [3]:
torch.cuda.set_device(0)

In [4]:
import model_RNN as model #this changed
import Loaddata_final as Loaddata
import TrVaTe as TVT #This changed 

# check GPU availability
use_cuda = torch.cuda.is_available()
use_cuda

True

In [5]:
# Load data set and target values
set_x = pickle.load(open('Data/h143.visits', 'rb'), encoding='bytes')
set_y = pickle.load(open('Data/h143.labels', 'rb'),encoding='bytes')

"""
model_x = []
for patient in set_x:
    model_x.append([each for visit in patient for each in visit])  
    
"""
model_x = set_x  #this is for the rest of the models
    
merged_set= [[set_y[i],model_x[i]] for i in range(len(set_y))] #list of list or list of lists of lists
print("\nLoading and preparing data...")    
train1, valid1, test1 = Loaddata.load_data(merged_set)
print("\nSample data after split:")  
print(train1[0])
print("model is", 'RNN') #can change afterwards, currently on most basic RNN


Loading and preparing data...

Sample data after split:
[0, [[1667, 144, 62, 85], [1667, 144, 62, 85]]]
model is RNN


In [6]:
def print2file(buf, outFile):
    outfd = open(outFile, 'a')
    outfd.write(buf + '\n')
    outfd.close()

logFile='JustRNN.log'
header = 'Model|EmbSize|CellType|n_Layers|Hidden|Dropout|Optimizer|LR|L2|EPs|BestValidAUC|TestAUC|atEpoch'
print2file(header, logFile)

In [7]:
epochs = 100

In [8]:
#test on RNN with celltype plain RNN, adamax 
def model_tune(embdim_exp, hid_exp, layers_n, dropout, l2_exp , lr_exp, eps_exp):
    #little transformations to use the searched values
    embed_dim = 2** int(embdim_exp)
    hidden_size = 2** int(hid_exp)
    n_layers = int(layers_n)
    dropout = round(dropout, 4)
    l2 = np.exp(l2_exp)
    lr = np.exp(lr_exp)
    eps = np.exp(eps_exp)

        
    """
   #dealing with categorical data
    if int(dlm_code)<3:
      if int(ct_code) ==1:
          cell_type='RNN'   
      elif int(ct_code) ==2:
          cell_type='LSTM'
      elif int(ct_code) ==3:
          cell_type='GRU'
      
    if int(dlm_code)==1:
        w_model='RNN'
        ehr_model = model.EHR_RNN(20000, embed_dim, hidden_size, n_layers, dropout, cell_type)
    elif int(dlm_code)==2:
        w_model='DRNN'
        ehr_model = model.DRNN(20000, embed_dim, hidden_size, n_layers, dropout, cell_type)

    if use_cuda:
        ehr_model = ehr_model.cuda(0)    
        
        
    if int(opt_code) ==1:
        opt= 'Adadelta'
        optimizer = optim.Adadelta(ehr_model.parameters(), lr=lr, weight_decay=l2 ,eps=eps) ## rho=0.9
    elif int(opt_code) ==2:
        opt= 'Adagrad'
        optimizer = optim.Adagrad(ehr_model.parameters(), lr=lr, weight_decay=l2) ##lr_decay no eps
    elif int(opt_code) ==3:
        opt= 'Adam'
        optimizer = optim.Adam(ehr_model.parameters(), lr=lr, weight_decay=l2,eps=eps ) ## Beta defaults (0.9, 0.999), amsgrad (false)
    elif int(opt_code) ==4:
        opt= 'Adamax'
        optimizer = optim.Adamax(ehr_model.parameters(), lr=lr, weight_decay=l2 ,eps=eps) ### Beta defaults (0.9, 0.999)
    elif int(opt_code) ==5:
        opt= 'RMSprop'
        optimizer = optim.RMSprop(ehr_model.parameters(), lr=lr, weight_decay=l2 ,eps=eps)                
    elif int(opt_code) ==6:
        opt= 'ASGD'
        optimizer = optim.ASGD(ehr_model.parameters(), lr=lr, weight_decay=l2 ) ### other parameters
    elif int(opt_code) ==7:
        opt= 'SGD'
        optimizer = optim.SGD(ehr_model.parameters(), lr=lr, weight_decay=l2 ) ### other parameters
  
    """
    ehr_model = model.EHR_RNN(20000, embed_dim, hidden_size, n_layers, dropout, cell_type = 'RNN')

    if use_cuda:
        ehr_model = ehr_model.cuda(0)    
    optimizer = optim.Adamax(ehr_model.parameters(), lr=lr, weight_decay=l2,eps=eps)         

     
    bestValidAuc = 0.0
    bestTestAuc = 0.0
    bestValidEpoch = 0
  
    for ep in range(epochs):
        current_loss, train_loss = TVT.train(train1, model= ehr_model, optimizer = optimizer, batch_size = 200)
        avg_loss = np.mean(train_loss)
        valid_auc, y_real, y_hat  = TVT.calculate_auc(model = ehr_model, data = valid1, which_model = 'RNN', batch_size = 200)
        if valid_auc > bestValidAuc: 
            bestValidAuc = valid_auc
            bestValidEpoch = ep
            best_model= ehr_model
            #bestTestAuc, y_real, y_hat = TVT.calculate_auc(model = ehr_model, data = test1, which_model = w_model, batch_size = 200)

        if ep - bestValidEpoch > 12:
            break
      
  
     
    fname= 'RNN'+'E'+str(embed_dim)+'RNN'+'L'+str(n_layers)+'H'+str(hidden_size)+'D'+str(dropout)+'Adamax'+'L'+str(lr)+'P'+str(l2)  
    bmodel_pth='models/'+fname
    bestTestAuc, y_real, y_hat = TVT.calculate_auc(model = best_model, data = test1, which_model = 'RNN', batch_size = 200)
    torch.save(best_model, bmodel_pth)
    buf = '|%f |%f |%d ' % (bestValidAuc, bestTestAuc, bestValidEpoch )
    
    pFile= 'RNN'+'|'+str(embdim_exp)+'|'+'RNN'+'|'+str(layers_n)+'|'+str(hid_exp)+'|'+str(dropout)+'|'+'Adamax'+'|'+str(lr_exp)+'|'+str(l2_exp)+'|'+str(eps_exp)+ buf  
    print2file(pFile, logFile)
    
    return bestValidAuc

In [9]:
init_df = pd.read_csv('BORNN_Init.csv' )
print(init_df.columns)
init_df.embdim_exp =np.log2(init_df.embdim_exp)
init_df.hid_exp = np.log2(init_df.hid_exp)
init_df.layers_n = init_df.layers_n.astype('float')
init_df.l2_exp = np.log(init_df.l2_exp)
init_df.lr_exp = np.log(init_df.lr_exp)
init_df.eps_exp = np.log(init_df.eps_exp)
init_df.head()

Index(['target', 'embdim_exp', 'hid_exp', 'layers_n', 'dropout', 'l2_exp',
       'lr_exp', 'eps_exp'],
      dtype='object')


,target,embdim_exp,hid_exp,layers_n,dropout,l2_exp,lr_exp,eps_exp
0,0.675914,8.0,8.0,1.0,0.1000,-3.000000,-3.000000,-6.000000
1,0.665504,5.0,6.0,1.0,0.1136,-5.052670,-4.886371,-5.882410
2,0.695596,8.0,5.0,1.0,0.2724,-3.518894,-4.147615,-5.483880
3,0.555088,6.0,8.0,1.0,0.6600,-1.362262,-2.489984,-8.804755
4,0.495318,6.0,7.0,2.0,0.0607,-1.706429,-2.728325,-5.999451


In [ ]:
# I might need to get rid of the loop now, if I want to put initialized point into them, let't just test plain RNN with Adamax
if __name__ == "__main__":
    gp_params = {"alpha": 1e-4}

#Here we loop through different models, change the model tune 
#for cti in range(1,4): 
    #for dlmi in range(1,2): #just the RNN, no DRNN
        #for opti in range(1,8):
            #print('\n Now Tuning model with Bayesian Optimization: ','cell code', str(cti),'model code', str(dlmi),'optimizer code',str(opti))
NNBO = BayesianOptimization(model_tune,
                            {#'dlm_code':(dlmi,dlmi), 'ct_code': (cti, cti), 'opt_code':(opti, opti),
                             'embdim_exp': (5, 9),'hid_exp': (5, 9),'layers_n': (1, 3),'dropout': (0, 1),
                             'l2_exp': (-6, -1), 'lr_exp': (-7, -2), 'eps_exp': (-9, -5)})

#add previously explored points to initialized points 
NNBO.initialize_df(init_df)
'''
 {
    'target': [-1166.19102, -1142.71370, -1138.68293],
    'alpha': [7.0034, 6.6186, 6.0798],
    'colsample_bytree': [0.6849, 0.7314, 0.9540],
    'gamma': [8.3673, 3.5455, 2.3281],
}

param points_df:
pandas dataframe with columns (target, {list of columns matching
self.keys})

ex:
      target        alpha      colsample_bytree        gamma
-1166.19102       7.0034                0.6849       8.3673
-1142.71370       6.6186                0.7314       3.5455
-1138.68293       6.0798                0.9540       2.3281
-1146.65974       2.4566                0.9290       0.3456
-1160.32854       1.9821                0.5298       8.7863

'''   


NNBO.explore({#'dlm_code':[dlmi], 'ct_code': [cti], 'opt_code':[opti],
             'embdim_exp': [8],'hid_exp': [8],'layers_n': [1],'dropout': [0.1],
              'l2_exp': [-3], 'lr_exp': [-3], 'eps_exp':[-6]}, eager = True)


NNBO.maximize(n_iter= 10000, **gp_params)

print('-' * 53)
print('Final Results')
print('RNN / DRNN: %f' % NNBO.res['max']['max_val'])

print2file(str(NNBO.res['max']), logFile)


Initialization
---------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   dropout |   embdim_exp |   eps_exp |   hid_exp |    l2_exp |   layers_n |    lr_exp | 
    1 | 11m51s |    0.65057 |    0.1000 |       8.0000 |   -6.0000 |    8.0000 |   -3.0000 |     1.0000 |   -3.0000 | 
Initialization
---------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   dropout |   embdim_exp |   eps_exp |   hid_exp |    l2_exp |   layers_n |    lr_exp | 
    2 | 10m48s |    0.53969 |    0.8738 |       5.7305 |   -5.1428 |    7.6278 |   -1.7097 |     1.3937 |   -2.3070 | 
    3 | 10m33s |    0.58751 |    0.5243 |       5.2544 |   -8.6173 |    8.6280 |   -2.2143 |     1.1681 |   -6.4372 | 
    4 | 12m02s |    0.59175 |    0.0118 |       8.4797 |   -6.6592 |    8.8715 |   -2.2138 |     2.1839 |   -5.2764 | 
    5 | 10m00s |    

  158 | 00m00s |    0.68370 |    0.0000 |       5.0000 |   -5.0000 |    9.0000 |   -6.0000 |     3.0000 |   -2.0000 | 
  159 | 00m00s |    0.69418 |    0.0000 |       7.0000 |   -7.6238 |    8.0000 |   -6.0000 |     3.0000 |   -2.0000 | 
  160 | 00m00s |    0.68251 |    0.0000 |       9.0000 |   -7.1880 |    7.0000 |   -6.0000 |     1.0000 |   -2.0000 | 
  161 | 00m00s |    0.66369 |    0.0000 |       5.0000 |   -5.0000 |    5.0000 |   -6.0000 |     1.0000 |   -5.2222 | 
  162 | 00m00s |    0.68654 |    0.0000 |       9.0000 |   -5.0000 |    5.0000 |   -3.5124 |     1.0000 |   -4.4816 | 
repeated
  163 | 00m00s |    0.66871 |    0.0000 |       9.0000 |   -5.0000 |    5.0000 |   -6.0000 |     3.0000 |   -2.0000 | 
  164 | 00m00s |    0.50000 |    1.0000 |       9.0000 |   -9.0000 |    6.0000 |   -5.2685 |     3.0000 |   -2.0000 | 
  165 | 00m00s |    0.63979 |    0.0000 |       9.0000 |   -9.0000 |    9.0000 |   -1.0000 |     1.0000 |   -5.8840 | 
  166 | 00m00s |    0.60108 |    0.0000

/home/jzhu8/.local/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'funcalls': 48, 'nit': 5, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00014627]), 'warnflag': 2}
  " state: %s" % convergence_dict)


  211 | 10m07s |    0.64592 |    1.0000 |       6.7880 |   -6.5072 |    9.0000 |   -3.6473 |     1.0000 |   -2.0000 | 
  212 | 19m04s |    0.70695 |    0.0052 |       8.6842 |   -8.8354 |    7.8937 |   -5.1064 |     2.9930 |   -5.5228 | 


/home/jzhu8/.local/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'funcalls': 47, 'nit': 4, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.00032569]), 'warnflag': 2}
  " state: %s" % convergence_dict)


...

  294 | 15m09s |    0.66466 |    1.0000 |       9.0000 |   -5.0000 |    9.0000 |   -6.0000 |     1.0000 |   -2.0000 | 
  295 | 11m33s |    0.69333 |    0.0000 |       9.0000 |   -7.2633 |    9.0000 |   -6.0000 |     1.9968 |   -5.4874 | 
  296 | 17m25s |    0.70056 |    0.9582 |       7.0725 |   -7.0179 |    7.5335 |   -5.8976 |     1.0000 |   -6.2928 | 
  297 | 21m18s |    0.69526 |    0.1120 |       8.9989 |   -5.1183 |    7.5400 |   -3.4219 |     1.0064 |   -6.9847 | 
  298 | 21m24s |    0.71019 |    0.8268 |       8.8904 |   -5.3967 |    8.5838 |   -4.0255 |     1.0000 |   -5.5484 | 
  299 | 14m30s |    0.69953 |    0.9506 |       7.9186 |   -6.9406 |    7.5323 |   -4.6780 |     1.0000 |   -5.4461 | 


/home/jzhu8/.local/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'funcalls': 51, 'nit': 3, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([1.40503425e-05]), 'warnflag': 2}
  " state: %s" % convergence_dict)


  300 | 25m41s |    0.70398 |    0.0000 |       9.0000 |   -8.0320 |    9.0000 |   -3.8510 |     1.0000 |   -6.2625 | 
  301 | 43m53s |    0.69904 |    0.5600 |       8.2432 |   -5.0000 |    6.9343 |   -3.9727 |     1.0000 |   -7.0000 | 
  302 | 28m59s |    0.68139 |    0.0000 |       9.0000 |   -9.0000 |    5.0000 |   -6.0000 |     1.0000 |   -3.5051 | 
  303 | 29m43s |    0.72027 |    0.0000 |       9.0000 |   -9.0000 |    9.0000 |   -3.7377 |     2.9646 |   -7.0000 | 
  304 | 13m55s |    0.69028 |    0.0000 |       8.8041 |   -9.0000 |    6.5266 |   -5.4608 |     2.4941 |   -4.5788 | 
  305 | 14m22s |    0.69573 |    1.0000 |       9.0000 |   -5.0000 |    9.0000 |   -5.2063 |     1.0000 |   -5.5004 | 
  306 | 18m02s |    0.69963 |    0.0000 |       7.1030 |   -6.6500 |    7.2623 |   -6.0000 |     1.0000 |   -4.6531 | 
  307 | 16m17s |    0.67360 |    1.0000 |       9.0000 |   -5.0000 |    8.7148 |   -3.3439 |     1.0000 |   -4.4013 | 
Error in iteration: 101, ignore result
Error in 

/home/jzhu8/.local/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'funcalls': 44, 'nit': 1, 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([7.78166567e-05]), 'warnflag': 2}
  " state: %s" % convergence_dict)


  310 | 43m48s |    0.68553 |    0.0169 |       8.9506 |   -6.9853 |    5.1288 |   -3.2344 |     1.0923 |   -5.3690 | 
